In [22]:
from src.data_handler import CoinDataStore
from src.simple_wash_detector_utils import detect_wash_trades_nearest

In [23]:
store = CoinDataStore("ETH", engine="fastparquet")
time_diff_s = 10 * 60 # 10 minutes difference
price_diff_pct = 0.01
size_diff_pct = 0.01

df_detected = detect_wash_trades_nearest(store, time_diff_s, price_diff_pct, size_diff_pct)
df_detected

,wallet_id,day,open_time,close_time,duration_s,open_side,close_side,open_price,close_price,price_change_pct,open_size,close_size,size_change_pct,pairing_direction,pair_id
0,2,2025-03-22,2025-03-22 12:58:34.300,2025-03-22 13:04:14.850,340.55,buy,sell,1994.300049,1994.800049,0.000251,1.0032,1.0032,0.0,buy_to_sell,0
1,2,2025-03-22,2025-03-22 13:05:00.900,2025-03-22 13:13:38.450,517.55,buy,sell,1993.400024,1994.199951,0.000401,1.0032,1.0032,0.0,buy_to_sell,1
2,2,2025-03-22,2025-03-22 13:28:46.350,2025-03-22 13:28:58.950,12.60,sell,buy,1988.199951,1988.800049,0.000302,1.0032,1.0032,0.0,sell_to_buy,2
3,2,2025-03-22,2025-03-22 13:28:58.950,2025-03-22 13:34:11.300,312.35,buy,sell,1988.800049,1989.699951,0.000452,1.0032,1.0032,0.0,buy_to_sell,3
4,2,2025-03-22,2025-03-22 13:34:14.750,2025-03-22 13:36:07.650,112.90,sell,buy,1989.699951,1986.900024,0.001407,1.0032,1.0032,0.0,sell_to_buy,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2016028,240638,2025-07-27,2025-07-27 01:57:33.650,2025-07-27 01:58:22.000,48.35,sell,buy,3758.899902,3760.699951,0.000479,0.0053,0.0053,0.0,sell_to_buy,2016028
2016029,240642,2025-07-27,2025-07-27 01:38:01.450,2025-07-27 01:38:04.150,2.70,buy,sell,3770.900146,3770.800049,0.000027,0.3222,0.3222,0.0,buy_to_sell,2016029
2016030,240664,2025-07-27,2025-07-27 01:57:41.800,2025-07-27 01:58:15.700,33.90,buy,sell,3759.000000,3759.199707,0.000053,0.3285,0.3285,0.0,buy_to_sell,2016030
2016031,240698,2025-07-27,2025-07-27 07:36:30.500,2025-07-27 07:38:17.850,107.35,buy,sell,3778.399658,3776.300049,0.000556,0.1583,0.1583,0.0,buy_to_sell,2016031


In [35]:
(df_detected["open_size"] * df_detected["open_price"]).sum() * 2

np.float32(39173284000.0)

In [30]:
store = CoinDataStore("ETH", engine="fastparquet")
df_all = store.load_all()
df_all

,price,size,time,seller,buyer
0,1996.500000,0.0075,2025-03-22 10:50:22.049676014,64,19509
1,1996.500000,0.0062,2025-03-22 10:50:22.049676014,64,19509
2,1996.500000,8.0000,2025-03-22 10:50:22.049676014,64,134
3,1996.500000,0.2212,2025-03-22 10:50:22.930512310,33,10024
4,1996.400024,1.0000,2025-03-22 10:50:23.052200940,18229,564
...,...,...,...,...,...
27785389,3776.600098,0.0031,2025-07-27 08:45:41.864000000,54585,821
27785390,3776.699951,0.0105,2025-07-27 08:45:43.018000000,49854,122768
27785392,3776.699951,0.0027,2025-07-27 08:45:45.058000000,49854,7559
27785391,3776.699951,0.2289,2025-07-27 08:45:45.058000000,49838,7559


In [31]:
df_all["size"].sum() / 10**6

np.float32(96.17093)

In [32]:
df_detected["open_size"].sum() * 2 / df_all["size"].sum()

np.float32(0.17109695)

In [9]:
import pandas as pd
import numpy as np


def detected_to_dfwash_full(df_detected: pd.DataFrame) -> pd.DataFrame:
    # core dfwash schema
    mapped = pd.DataFrame({
        "wallet_id": df_detected["wallet_id"],
        "price1": df_detected["open_price"],
        "size1": df_detected["open_size"],
        "price2": df_detected["close_price"],
        "size2": df_detected["close_size"],
        "side1": df_detected["open_side"],
        "side2": df_detected["close_side"],
        "t1": pd.to_datetime(df_detected["open_time"], errors="coerce"),
        "t2": pd.to_datetime(df_detected["close_time"], errors="coerce"),
        "delta_seconds": df_detected["duration_s"],
    }, index=df_detected.index)

    # size ratio
    mapped["size_ratio"] = mapped["size2"] / mapped["size1"].replace(0, np.nan)

    # enforce lightweight dtypes
    cast = {
        "wallet_id": "uint32",
        "price1": "float32", "size1": "float32",
        "price2": "float32", "size2": "float32",
        "delta_seconds": "float32", "size_ratio": "float32",
    }
    for c, dt in cast.items():
        if c in mapped.columns:
            try:
                mapped[c] = mapped[c].astype(dt, copy=False)
            except Exception:
                pass

    # derived cols
    mapped["direction"] = (mapped["side1"].astype("string") + "->" + mapped["side2"].astype("string")).astype("category")
    mapped["dt_s"] = mapped["delta_seconds"].astype("float32")
    mapped["size_err_pct"] = (mapped["size_ratio"] - 1.0).abs().astype("float32") * 100.0
    base = mapped["price1"].replace(0, np.nan)
    mapped["price_change_pct"] = ((mapped["price2"] - mapped["price1"]) / base).astype("float32")
    mapped["abs_price_change_bps"] = (mapped["price_change_pct"].abs() * 1e4).astype("float32")
    mapped["same_price"] = (mapped["price2"] - mapped["price1"]).abs() <= 1e-8
    mapped["date"] = mapped["t1"].dt.date
    mapped["hour"] = mapped["t1"].dt.hour.astype("int16")
    mapped["dow"] = mapped["t1"].dt.dayofweek.astype("int8")

    # combine original df_detected + mapped features
    df_full = pd.concat([df_detected.reset_index(drop=True), mapped.reset_index(drop=True)], axis=1)
    # remove duplicated column names across the whole frame
    df_full = df_full.loc[:, ~df_full.columns.duplicated()]

    return df_full


In [10]:
from src.wash_trading_pairs_analyser import make_dfwash_plots, global_snapshot, per_wallet_leaderboard, \
    plot_wash_notional_by_date_bar, plot_wash_volume_by_date_bar

# If you have the DataFrame already:
token = "HYPE"

def wash_trading_pairs_analysis(dfwash: pd.DataFrame, token: str, detector_type: str):
    dfwash = detected_to_dfwash_full(df_detected)

    make_dfwash_plots(dfwash, out_dir=f"analysis/{token}/{detector_type}/plots_dfwash_general", show=True, top_wallets=25)
    global_snapshot(dfwash, out_dir=f"analysis/{token}/{detector_type}/", fname=f"global_snapshot_{token}.json")

    per_wallet_leaderboard(dfwash, top=50, out_dir=f"analysis/{token}/{detector_type}/")
    plot_wash_volume_by_date_bar(dfwash, freq="D", out_dir=f"analysis/{token}/{detector_type}/", show=False)
    plot_wash_notional_by_date_bar(dfwash, freq="D", out_dir=f"analysis/{token}/{detector_type}/", show=False)

In [17]:
dfwash = detected_to_dfwash_full(df_detected)

In [18]:
df_all.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1478247 entries, 0 to 1478246
Data columns (total 5 columns):
 #   Column  Non-Null Count    Dtype         
---  ------  --------------    -----         
 0   price   1478247 non-null  float32       
 1   size    1478247 non-null  float32       
 2   time    1478247 non-null  datetime64[ns]
 3   seller  1478247 non-null  uint64        
 4   buyer   1478247 non-null  uint64        
dtypes: datetime64[ns](1), float32(2), uint64(2)
memory usage: 56.4 MB


In [19]:
import pandas as pd

def wash_trade_summary(df_trades, df_wash):
    """
    Compare df_trades with df_wash and return summary stats.

    Returns a DataFrame with:
      - total trades count
      - total volume (price * size)
      - percentage of trades/volume
      - mean price and size
    """
    # Compute total volume for all trades
    df_trades['trade_volume'] = df_trades['price'] * df_trades['size']

    # Flatten df_wash into individual trades
    wash_buyers = df_wash[['t1', 'price1', 'size1', 'side1', 'pair_id']].rename(
        columns={'t1':'time', 'price1':'price', 'size1':'size', 'side1':'side'}
    )
    wash_sellers = df_wash[['t2', 'price2', 'size2', 'side2', 'pair_id']].rename(
        columns={'t2':'time', 'price2':'price', 'size2':'size', 'side2':'side'}
    )

    df_wash_trades = pd.concat([wash_buyers, wash_sellers], ignore_index=True)
    df_wash_trades['trade_volume'] = df_wash_trades['price'] * df_wash_trades['size']

    # Total stats
    total_trades = len(df_trades)
    total_volume = df_trades['trade_volume'].sum()

    # Wash trades stats
    wash_trades_count = len(df_wash_trades)
    wash_volume = df_wash_trades['trade_volume'].sum()

    # Non-wash trades stats
    non_wash_count = total_trades - wash_trades_count
    non_wash_volume = total_volume - wash_volume

    # Average price and size
    avg_price_wash = df_wash_trades['price'].mean()
    avg_size_wash = df_wash_trades['size'].mean()
    avg_price_non = df_trades[~df_trades.index.isin(df_wash_trades.index)]['price'].mean()
    avg_size_non = df_trades[~df_trades.index.isin(df_wash_trades.index)]['size'].mean()

    # Build summary DataFrame
    summary = pd.DataFrame({
        'Category': ['Wash Trades', 'Non-Wash Trades', 'Total'],
        'Count': [wash_trades_count, non_wash_count, total_trades],
        'Count %': [wash_trades_count/total_trades*100,
                    non_wash_count/total_trades*100, 100],
        'Volume': [wash_volume, non_wash_volume, total_volume],
        'Volume %': [wash_volume/total_volume*100,
                     non_wash_volume/total_volume*100, 100],
        'Avg Price': [avg_price_wash, avg_price_non, df_trades['price'].mean()],
        'Avg Size': [avg_size_wash, avg_size_non, df_trades['size'].mean()]
    })

    return summary


In [21]:
summary_df = wash_trade_summary(df_all, dfwash)
print(summary_df)


          Category    Count     Count %        Volume    Volume %  Avg Price  \
0      Wash Trades   119506    8.084305  1.574069e+08    9.202271  20.916893   
1  Non-Wash Trades  1358741   91.915695  1.553116e+09   90.797737  21.080706   
2            Total  1478247  100.000000  1.710522e+09  100.000000  21.024830   

    Avg Size  
0  62.526768  
1  54.999031  
2  55.354176  
